In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Callable, Dict, Iterable, Literal, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qresearch.data.io import load_hsci_2008_components, load_hsci_components_history
from qresearch.data.utils import get_processed_dir, save_market_data_to_csv, load_market_data_from_csv
from qresearch.data.yfinance import download_market_data

In [34]:
# ============================================================
# 0) Core utilities: naming + label/feature building
# ============================================================
def ensure_names(close_df: pd.DataFrame) -> pd.DataFrame:
    close = close_df.sort_index().copy()
    close.index = pd.to_datetime(close.index)
    close.index.name = "date"
    close.columns.name = "ticker"
    return close


def make_fwd_return(close_df: pd.DataFrame, H: int = 5) -> pd.DataFrame:
    close = ensure_names(close_df)
    return close.shift(-H) / close - 1.0


def rsi_wilder(close: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    delta = close.diff()
    up = delta.clip(lower=0.0)
    down = (-delta).clip(lower=0.0)
    roll_up = up.ewm(alpha=1 / period, adjust=False).mean()
    roll_down = down.ewm(alpha=1 / period, adjust=False).mean()
    rs = roll_up / roll_down
    return 100.0 - (100.0 / (1.0 + rs))


def make_features_default(close_df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    close = ensure_names(close_df)

    ma5 = close.rolling(5).mean()
    ma10 = close.rolling(10).mean()
    ma20 = close.rolling(20).mean()

    close_ma_diff_5 = close / ma5 - 1.0
    close_ma_diff_10 = close / ma10 - 1.0
    close_ma_diff_20 = close / ma20 - 1.0

    max_growth_5 = close / close.shift(5) - 1.0

    rsi = rsi_wilder(close, period=14)

    roll_max_120 = close.rolling(120).max()
    dd = close / roll_max_120 - 1.0
    max_drawdown_120 = dd.rolling(120).min()

    return {
        "close_ma_diff_5": close_ma_diff_5,
        "max_growth_5": max_growth_5,
        "close_ma_diff_10": close_ma_diff_10,
        "rsi": rsi,
        "close_ma_diff_20": close_ma_diff_20,
        "max_drawdown_120": max_drawdown_120,
    }


def wide_to_panel(features: Dict[str, pd.DataFrame], y_wide: pd.DataFrame) -> pd.DataFrame:
    """
    Convert wide feature dict + wide y into a long panel:
    columns: date, ticker, <feature...>, y
    """
    parts = []
    for name, df in features.items():
        s = df.stack(future_stack=True).rename(name)
        parts.append(s)

    X = pd.concat(parts, axis=1)
    y = y_wide.stack(future_stack=True).rename("y")

    panel = pd.concat([X, y], axis=1)
    panel.index = panel.index.set_names(["date", "ticker"])
    panel = panel.reset_index()
    panel["date"] = pd.to_datetime(panel["date"])
    return panel


# ============================================================
# 1) Cross-sectional preprocessing
# ============================================================
def cs_winsorize_zscore(panel: pd.DataFrame, feature_cols: list[str], q_lo: float = 0.01, q_hi: float = 0.99) -> pd.DataFrame:
    out = panel.copy()
    for c in feature_cols:
        g = out.groupby("date")[c]
        lo = g.transform(lambda s: s.quantile(q_lo))
        hi = g.transform(lambda s: s.quantile(q_hi))
        x = out[c].clip(lo, hi)
        mu = x.groupby(out["date"]).transform("mean")
        sd = x.groupby(out["date"]).transform("std")
        out[c] = (x - mu) / sd
        
    return out


def flip_negative_ic(panel: pd.DataFrame, feature_cols: list[str]) -> Tuple[pd.DataFrame, Dict[str, float]]:
    """
    Pooled Spearman IC between each feature and y.
    If IC < 0, flip sign to enforce positive direction.
    """
    out = panel.copy()
    ic_map: Dict[str, float] = {}
    for c in feature_cols:
        ic = out[c].corr(out["y"], method="spearman")
        ic_map[c] = float(ic) if ic is not None else np.nan
        if ic is not None and ic < 0:
            out[c] = -out[c]
    return out, ic_map


# ============================================================
# 2) Walk-forward split (2y train / 1m test + purge H)
# ============================================================
def walk_forward_splits(
    dates: Iterable[pd.Timestamp],
    train_years: int = 2,
    test_months: int = 1,
    H: int = 5,
    min_train_days: int = 100,
    min_test_days: int = 5,
):
    dates = pd.DatetimeIndex(sorted(pd.DatetimeIndex(dates).unique()))
    start = dates.min()
    end = dates.max()

    cur_test_start = start + pd.DateOffset(years=train_years)
    while cur_test_start < end:
        cur_test_end = cur_test_start + pd.DateOffset(months=test_months)

        test_dates = dates[(dates >= cur_test_start) & (dates < cur_test_end)]
        if len(test_dates) == 0:
            cur_test_start = cur_test_end
            continue

        test_start = test_dates.min()
        pos = dates.get_indexer([test_start])[0]
        train_end_pos = max(pos - H, 0)  # purge H days
        train_end = dates[train_end_pos]

        train_start = test_start - pd.DateOffset(years=train_years)
        train_dates = dates[(dates >= train_start) & (dates <= train_end)]

        if len(train_dates) >= min_train_days and len(test_dates) >= min_test_days:
            yield train_dates, test_dates

        cur_test_start = cur_test_end


# ============================================================
# 3) Model adapters (single interface)
# ============================================================
@dataclass(frozen=True)
class FitPredictContext:
    feature_cols: list[str]
    top_k: int
    H: int


FitPredictFn = Callable[
    [pd.DataFrame, pd.DataFrame, np.ndarray, np.ndarray, np.ndarray, np.ndarray, FitPredictContext, dict],
    np.ndarray
]
"""
Signature:
    predict = fit_predict(
        train_df, test_df,
        X_train, y_train, group_train,
        X_test, group_test,
        ctx, params
    )
"""

def make_relevance_from_y(
    panel: pd.DataFrame,
    n_bins: int = 5,
    col_y: str = "y",
    col_date: str = "date",
    min_per_day: int = 20,
) -> pd.Series:
    """
    Convert continuous forward returns y into integer relevance labels per day.

    Why:
    - LightGBM LambdaRank expects integer labels (relevance grades).
    - Relevance must be cross-sectional per query group (here: per date).

    Robustness:
    - Handles ties / insufficient unique values / small universes.
    - Guarantees output is integer in [0, n_bins-1] where defined.
    - Returns pd.Series aligned to panel.index (same length).

    Convention:
    - Higher y => higher relevance (better).
    """
    if n_bins < 2:
        raise ValueError("n_bins must be >= 2")

    y = panel[col_y]
    d = panel[col_date]

    def _bin_one_day(s: pd.Series) -> pd.Series:
        # s is y for one date, indexed by panel index rows
        s = s.replace([np.inf, -np.inf], np.nan).dropna()

        # Not enough names to create stable bins
        if len(s) < max(min_per_day, n_bins):
            return pd.Series(index=s.index, data=np.nan)

        # If too many ties (e.g., many zeros), qcut can fail or drop bins
        # Strategy: rank then map ranks into bins deterministically.
        r = s.rank(method="average")  # ascending rank: low y -> small rank
        # Convert rank percentiles to bins
        # bin_id in 0..n_bins-1
        bin_id = np.floor((r - 1) / max(len(r) / n_bins, 1.0))
        bin_id = bin_id.clip(0, n_bins - 1)

        # Ensure integer dtype but allow NaN at higher level; return Int64 for safety
        return pd.Series(index=s.index, data=bin_id.astype(np.int64))

    rel = panel.groupby(d, sort=False)[col_y].apply(_bin_one_day)
    # groupby/apply returns a multiindex series; drop the group level so it aligns with original rows
    rel = rel.reset_index(level=0, drop=True)

    # Align to full panel index: rows that were NaN/dropped become NaN here
    rel = rel.reindex(panel.index)

    # Use pandas nullable Int64 to avoid int-casting NaN error.
    return rel.astype("Int64")


def fit_predict_xgb_ranker(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    X_train: np.ndarray,
    y_train: np.ndarray,
    group_train: np.ndarray,
    X_test: np.ndarray,
    group_test: np.ndarray,
    ctx: FitPredictContext,
    params: dict,
) -> np.ndarray:
    from xgboost import XGBRanker

    # map friendly params -> XGBRanker init
    model = XGBRanker(**params)
    model.fit(X_train, y_train, group=group_train)
    return model.predict(X_test)


def fit_predict_lgb_lambdarank(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    X_train: np.ndarray,
    y_train: np.ndarray,
    group_train: np.ndarray,
    X_test: np.ndarray,
    group_test: np.ndarray,
    ctx: FitPredictContext,
    params: dict,
) -> np.ndarray:
    import lightgbm as lgb

    # LightGBM needs Dataset + group
    dtrain = lgb.Dataset(X_train, label=y_train, group=group_train, free_raw_data=True)
    dvalid = lgb.Dataset(X_test,  label=np.zeros_like(X_test[:, 0]), group=group_test, reference=dtrain, free_raw_data=True)

    # training params
    lgb_params = params.get("lgb_params", {})
    num_boost_round = params.get("num_boost_round", 2000)
    early_stopping_rounds = params.get("early_stopping_rounds", 50)

    # ensure objective/metric exist
    lgb_params = {
        "objective": "lambdarank",
        "metric": "ndcg",
        "verbosity": -1,
        "ndcg_eval_at": [ctx.top_k],
        **lgb_params,
    }

    booster = lgb.train(
        lgb_params,
        dtrain,
        num_boost_round=num_boost_round,
        valid_sets=[dvalid],
        callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False)],
    )
    return booster.predict(X_test, num_iteration=booster.best_iteration)


MODEL_REGISTRY: Dict[str, FitPredictFn] = {
    "xgb_ranker": fit_predict_xgb_ranker,
    "lgb_lambdarank": fit_predict_lgb_lambdarank,
}


# ============================================================
# 4) Shared evaluation
# ============================================================
def eval_rankic(oos: pd.DataFrame) -> Dict[str, object]:
    rankic_by_day = oos.groupby("date", group_keys=False)[["score", "y"]].apply(lambda g: g["score"].corr(g["y"], method="spearman"))
    mean = float(rankic_by_day.mean())
    std = float(rankic_by_day.std(ddof=0))
    ir = float(mean / std) if std > 0 else np.nan
    return {"rankic_by_day": rankic_by_day, "rankic_mean": mean, "rankic_ir": ir}


def eval_topk_proxy(oos: pd.DataFrame, top_k: int) -> Dict[str, object]:
    # Diagnostic only: mean of forward H-day return y among top-k scores
    topk_fwd = oos.groupby("date", group_keys=False)[["score", "y"]].apply(lambda g: g.nlargest(top_k, "score")["y"].mean())
    topk_eq = (1 + topk_fwd.fillna(0)).cumprod()
    return {"topk_fwd": topk_fwd, "topk_fwd_eq": topk_eq}


# ============================================================
# 5) The unified workflow (minimal duplication)
# ============================================================
def run_rank_workflow(
    close_df: pd.DataFrame,
    model_name: Literal["xgb_ranker", "lgb_lambdarank"],
    model_params: dict,
    H: int = 5,
    top_k: int = 7,
    train_years: int = 2,
    test_months: int = 1,
    preprocess: bool = True,
    align_feature_sign: bool = True,
) -> Dict[str, object]:
    """
    Unified workflow for cross-sectional ranking models (walk-forward).

    Inputs
    ------
    close_df:
        Wide close price table: index=Date, columns=Ticker.
        Features and y are computed using only information up to close[t].
    model_name:
        Which ranker to use (delegated to MODEL_REGISTRY).
    model_params:
        Model-specific parameters, plus optional workflow knobs like:
          - relevance_bins (for lgb_lambdarank)
    H:
        Forward horizon: y[t] = fwd return over the holding window aligned to entry rule.
    top_k:
        Used by eval_topk_proxy (diagnostic), and can be used by the ranker objective.
    train_years / test_months:
        Walk-forward split definition.
    preprocess:
        Cross-sectional winsorize + zscore per date (recommended).
    align_feature_sign:
        Flip features with negative pooled IC so “higher is better” consistently.

    Returns
    -------
    dict with:
        - pooled_ic (optional)
        - oos (long panel of predictions)
        - oos_wide (date x ticker score table; directly usable as a signal)
        - rankic stats
        - topk proxy stats
        - metadata
    """
    if model_name not in MODEL_REGISTRY:
        raise KeyError(f"Unknown model_name={model_name}. Available: {list(MODEL_REGISTRY.keys())}")

    fit_predict = MODEL_REGISTRY[model_name]

    # -------------------------
    # 1) Build a single shared panel: (date, ticker) rows with features + y
    # -------------------------
    feats = make_features_default(close_df)
    y_wide = make_fwd_return(close_df, H=H)

    panel = wide_to_panel(feats, y_wide)  # expects columns: date, ticker, <features...>, y

    feature_cols = list(feats.keys())

    # Hard clean y to avoid inf contaminating downstream ops
    # (You can optionally extend to features, but you already dropna later.)
    panel = panel[np.isfinite(panel["y"].to_numpy())].copy()

    # -------------------------
    # 2) LightGBM LambdaRank needs integer relevance grades per query group (date)
    # -------------------------
    if model_name == "lgb_lambdarank":
        n_bins = int(model_params.get("relevance_bins", 5))
        panel = panel.copy()
        panel["rel"] = make_relevance_from_y(panel, n_bins=n_bins, col_y="y", col_date="date")

    # -------------------------
    # 3) Drop NA rows from rolling windows + end-of-sample horizon
    # -------------------------
    drop_cols = ["y"] + feature_cols
    if model_name == "lgb_lambdarank":
        # rel is allowed to be NA for bad dates; but training cannot.
        drop_cols += ["rel"]

    panel = panel.dropna(subset=drop_cols)

    # -------------------------
    # 4) Optional cross-sectional preprocessing (per date)
    # -------------------------
    if preprocess:
        panel = cs_winsorize_zscore(panel, feature_cols)

    pooled_ic = None
    if align_feature_sign:
        panel, pooled_ic = flip_negative_ic(panel, feature_cols)

    # Ensure chronological order for splits
    all_dates = pd.DatetimeIndex(panel["date"].unique()).sort_values()

    ctx = FitPredictContext(feature_cols=feature_cols, top_k=top_k, H=H)

    # -------------------------
    # 5) Walk-forward: train -> predict -> collect OOS scores
    # -------------------------
    oos_rows = []

    for train_dates, test_dates in walk_forward_splits(
        all_dates,
        train_years=train_years,
        test_months=test_months,
        H=H,
    ):
        train = panel[panel["date"].isin(train_dates)].sort_values(["date", "ticker"])
        test  = panel[panel["date"].isin(test_dates)].sort_values(["date", "ticker"])

        # Guard: skip empty splits (can happen if dates get filtered by NA drops)
        if len(train) == 0 or len(test) == 0:
            continue

        # Group sizes: one query group per date
        group_train = train.groupby("date").size().to_numpy()
        group_test  = test.groupby("date").size().to_numpy()

        X_train = train[feature_cols].to_numpy()
        X_test  = test[feature_cols].to_numpy()

        # y is always continuous forward return (kept for evaluation)
        y_test = test["y"].to_numpy()

        # For training target:
        # - XGB ranker can train directly on continuous y (pairwise)
        # - LGBM lambdarank typically expects relevance grades (int)
        if model_name == "lgb_lambdarank":
            y_train_model = train["rel"].to_numpy()
        else:
            y_train_model = train["y"].to_numpy()

        score = fit_predict(
            train, test,
            X_train, y_train_model, group_train,
            X_test, group_test,
            ctx, model_params,
        )

        tmp = test[["date", "ticker"]].copy()
        tmp["score"] = score
        tmp["y"] = y_test
        oos_rows.append(tmp)

    if not oos_rows:
        raise RuntimeError("No OOS rows produced. Check splits/NA filtering/universe coverage.")

    oos = (
        pd.concat(oos_rows, ignore_index=True)
        .sort_values(["date", "score"], ascending=[True, False])
    )

    # Wide score table: convenient “signal” to plug into bucket backtest / tearsheet
    oos_wide = oos.pivot(index="date", columns="ticker", values="score").sort_index()

    rank_metrics = eval_rankic(oos)
    topk_metrics = eval_topk_proxy(oos, top_k=top_k)

    return {
        "pooled_ic": pooled_ic,
        "oos": oos,
        "oos_wide": oos_wide,
        **rank_metrics,
        **topk_metrics,
        "model_name": model_name,
        "model_params": model_params,
        "H": H,
        "top_k": top_k,
        "train_years": train_years,
        "test_months": test_months,
        "preprocess": preprocess,
        "align_feature_sign": align_feature_sign,
    }


In [35]:
import yfinance as yf

processed_dir = get_processed_dir()

hsci_stocks = pd.read_csv(f'{processed_dir}/hsci_components.csv')

START_DATE = "2009-01-01"
END_DATE   = None

def code_int_to_hk(code: int) -> str:
    return f"{int(code):04d}.HK"

UNIVERSE = hsci_stocks["Stock Code"].apply(code_int_to_hk).tolist()

# price_df = download_market_data(UNIVERSE, start=START_DATE, end=END_DATE)

In [36]:
# save_market_data_to_csv(price_df, f'{processed_dir}/hsci_ohlc.csv')

In [37]:
hsci_md = load_market_data_from_csv(f'{processed_dir}/hsci_ohlc.csv')
hsci_md.close.head()

Loading MarketData from /Users/henrywzh/Desktop/Quant/Research/data/processed/hsci_ohlc.csv...


Ticker,0883.HK,0857.HK,1088.HK,0386.HK,1171.HK,1898.HK,2883.HK,3668.HK,2386.HK,0639.HK,...,1052.HK,9699.HK,2510.HK,0636.HK,1341.HK,2169.HK,2570.HK,3677.HK,2582.HK,1333.HK
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-02,2.767038,3.167000,5.170709,1.308784,0.736292,3.657679,4.749063,NaN,NaN,0.425364,...,0.829630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,2.985776,3.386929,5.349010,1.387944,0.807467,3.959227,5.000485,NaN,NaN,0.434272,...,0.854555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,2.989422,3.439713,5.473820,1.374751,0.830782,4.188180,4.853823,NaN,NaN,0.414229,...,0.868798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,2.880052,3.276964,5.444103,1.324616,0.809921,3.953643,4.679226,NaN,NaN,0.412002,...,0.858115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,2.686833,3.105419,5.259860,1.245456,0.742428,3.573914,4.211303,NaN,NaN,0.420910,...,0.811827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# hsi_md = download_market_data("^HSI", START_DATE, END_DATE)
# 
# # Extract close matrices
# close = hsci_md.close.sort_index()
# # close.to_csv(f'{grandparent}/data/processed/hsci_close.csv')
# benchmark_close = hsi_md.close.sort_index()
# 
# # Common calendar (optional but recommended)
# common_dates = close.index.intersection(benchmark_close.index)
# close_df = close.loc[common_dates]
# benchmark_close = benchmark_close.loc[common_dates]

In [39]:
# from qresearch.backtest.buckets import make_tearsheet
# 
# lgb_params = {
#   "relevance_bins": 5,
#   "lgb_params": {
#     "learning_rate": 0.05,
#     "num_leaves": 31,
#     "min_data_in_leaf": 300,
#     "feature_fraction": 0.8,
#     "bagging_fraction": 0.8,
#     "bagging_freq": 1,
#     "lambda_l2": 2.0,
#     "min_gain_to_split": 0.0,
#   },
#   "num_boost_round": 800,
# }
# 
# out = run_rank_workflow(
#     close_df=close_df,
#     model_name="lgb_lambdarank",
#     model_params=lgb_params,
#     H=5,
#     top_k=10,
# )
# 
# signal_ml = out["oos_wide"]  # date x ticker model scores
# 
# rep = make_tearsheet(
#     price_df=price_df,
#     signal=signal_ml,
#     H=5,
#     n_buckets=20,
#     entry_mode="next_close",
#     benchmark_price=hsi,  # optional
#     benchmark_name="^HSI",
# )

In [40]:
from qresearch.signals.signals_sweep import *

# H = 5

test_ma_diff = [
    ('ma_diff', {"lookback": 5, "skip": 0}),
    ('ma_diff', {"lookback": 20, "skip": 0}),
    ('ma_diff', {"lookback": 120, "skip": 0}),
    ('ma_diff', {"lookback": 200, "skip": 0}),
    ('ma_diff', {"lookback": 250, "skip": 0}),
]

test_mom_ret = [
    ("mom_ret", {"lookback": 5, "skip": 0}),
    ("mom_ret", {"lookback": 21, "skip": 0}),
    ("mom_ret", {"lookback": 126, "skip": 0}),
    ("mom_12_1", {"lookback": 252, "skip": 21}),
]

test_ohlc_mom = [
    ("on_minus_id", {"lookback": 5}),
    ("overnight_mom", {"lookback": 5}),
    ('intraday_mom', {"lookback": 5}),
    ("on_minus_id", {"lookback": 10}),
    ("overnight_mom", {"lookback": 10}),
    ('intraday_mom', {"lookback": 10}),
    ("on_minus_id", {"lookback": 21}),
    ("overnight_mom", {"lookback": 21}),
    ('intraday_mom', {"lookback": 21}),
    ("on_minus_id", {"lookback": 63}),
    ("overnight_mom", {"lookback": 63}),
    ('intraday_mom', {"lookback": 63}),
    ("on_minus_id", {"lookback": 126}),
    ("overnight_mom", {"lookback": 126}),
    ('intraday_mom', {"lookback": 126}),
    ("on_minus_id", {"lookback": 252}),
    ("overnight_mom", {"lookback": 252}),
    ('intraday_mom', {"lookback": 252}),
]

test_rsi = [
    ("rsi", {"lookback": 5}),
    ("rsi", {"lookback": 12}),
    ("rsi", {"lookback": 24}),
]

test_trend_annret_r2 = [
    ("trend_annret_r2", {"lookback": 126, "ann_factor": 252}),
    ("trend_annret_r2", {"lookback": 252, "ann_factor": 252}),
    ("trend_annret_r2", {"lookback": 252, "ann_factor": 252, 'skip': 21}),
]

tests = test_ma_diff + test_mom_ret + test_trend_annret_r2 + test_ohlc_mom + test_rsi

In [41]:
# cfg = SignalTestConfig(H=5, n_buckets=10, use_price_floor=True)
# summary = sweep_signals(hsci_md, tests, cfg)
# summary

In [42]:
# cfg = SignalTestConfig(H=10, n_buckets=10, use_price_floor=True)
# summary = sweep_signals(hsci_md, tests, cfg)
# summary

In [43]:
# cfg = SignalTestConfig(H=20, n_buckets=10, use_price_floor=True)
# summary = sweep_signals(hsci_md, tests, cfg)
# summary

## Market Cap, Market Data

In [46]:
from qresearch.universe.hsci import build_hsci_ticker_master

all_df = load_hsci_components_history()
hsci_2008_df = load_hsci_2008_components()

hsci_ticker_master = build_hsci_ticker_master(all_df, hsci_2008_df)

tickers = hsci_ticker_master['ticker']

hsci_ticker_master

,ticker,in_seed_2008,in_history_events
0,0001.HK,True,True
1,0002.HK,True,False
2,0003.HK,True,False
3,0004.HK,True,True
4,0005.HK,True,False
...,...,...,...
929,9993.HK,False,True
930,9995.HK,False,True
931,9996.HK,False,True
932,9997.HK,False,True


In [48]:
# ---- market data ----
hsci_all_md = download_market_data(tickers, start=START_DATE, end=END_DATE)

[                       0%                       ]  2 of 934 completed

[-] Initializing download for 934 tickers...
    Date Range: 2009-01-01 -> Now


[*********************100%***********************]  934 of 934 completed

65 Failed downloads:
['0735.HK', '0569.HK', '1821.HK', '0729.HK', '0378.HK', '2868.HK', '3799.HK', '2662.HK', '1131.HK', '0208.HK', '0342.HK', '0665.HK', '0985.HK', '1337.HK', '0996.HK', '6837.HK', '0054.HK', '2148.HK', '0903.HK', '1175.HK', '0885.HK', '1366.HK', '0109.HK', '0973.HK', '1589.HK', '1297.HK', '3308.HK', '1573.HK', '1212.HK', '1363.HK', '1992.HK', '1169.HK', '2686.HK', '1558.HK', '1031.HK', '0530.HK', '1381.HK', '0043.HK', '0678.HK', '1089.HK', '1806.HK', '0190.HK', '0416.HK', '2103.HK', '0925.HK', '9997.HK', '0494.HK', '0555.HK', '1194.HK', '1829.HK', '0680.HK', '0273.HK', '0958.HK', '0906.HK']: YFTzMissingError('possibly delisted; no timezone found')
['1136.HK', '1768.HK', '1688.HK', '0049.HK', '0349.HK', '0203.HK', '0773.HK', '2626.HK', '0930.HK', '2332.HK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2009-01-01 -> 2026-01-19)')
['2066.HK']: YFPricesMissingError('possibly de

[✓] Download complete. Shape: (4354, 4735)
[-] Extracting and aligning components...
[✓] MarketData object created..


In [49]:
# save_market_data_to_parquet(hsci_all_md, get_processed_dir() / 'market/hsci_ohlc.parquet')

In [1]:
# tickers = list(hsci_all_md.close.columns)
# 
# raw = download_mcap_snapshot_yf(tickers)
# snap = finalize_mcap_snapshot(raw, asof_date=hsci_all_md.close.index[-1])
# 
# shares = compute_estimated_shares(hsci_all_md.close, snap, px_asof=hsci_all_md.close.index[-1])
# mcap = build_historical_mcap(hsci_all_md.close, shares)


# save_mkt_cp_snapshot(snap, "data/processed/reference/hsci_mktcap_snapshot.parquet")

# save_estimated_shares(shares, "data/processed/reference/hsci_estimated_shares.parquet",
#                       asof_date=snap["asof_date"].iloc[0], px_asof_date=hsci_all_md.close.index[-1])


# save_historical_mcap_parquet(mcap, "data/processed/reference/hsci_mcap_approx.parquet")


In [54]:
# save_historical_mcap_parquet(historical_mcap, get_processed_dir() / 'fundamentals/hsci_mkt_cp.parquet')

PosixPath('/Users/henrywzh/Desktop/Quant/Research/data/processed/fundamentals/hsci_mkt_cap.parquet')

In [58]:
# save_estimated_shares(estimated_shares, get_processed_dir() / 'fundamentals/hsci_estimated_shares.parquet')

PosixPath('/Users/henrywzh/Desktop/Quant/Research/data/processed/fundamentals/hsci_estimated_shares.parquet')